# Preprocessing

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path


def rename_columns(df):
    # Patient Characteristics columns
    patient_characteristics = [
        "Patient number", "Age (Continuos)", "Sex (Male/female)", "BMI (Continuous",
        "Roken (Yes/No)", "Diabetes (Yes/No)", "Cardiovascular disease (Yes/No)",
        "Rheumatoid Arthritis(Yes/No)"
    ]

    # Fracture Characteristics columns
    fracture_characteristics = [
        "Treatment (Conservative/Operative)",
        "High Energy trauma (Yes/No)",
        "Trauma mechanism (Valgus-flexion, Valgus-extension, Valgus-hyperextension, Varus-flexion, Varus-extension, "
        "Varus-Hyperextension)",
        "Side Fracture fractuur (Right/Left)", "AO/OTA Classification ( 1-6)",
        "Posterior involvement (Yes/No)", "Fracture fibula head (Yes/No)",
        "Cruris Fracture (Yes/No)", "2D gap (Continuous, mm)", "2D Step-off (Continuous, mm)", "3DGap Area",
        "AO/OTA - Severtity"
    ]

    # Postop Characteristics columns
    postop_characteristics = [
        "Condylar width (Continuous, mm)", "Incongruence (Continuous, mm)",
        "MPTA(Continuous, degree)", "PPTA (Continuous, degree)",
        "Revision surgery (Yes/No)", "Complicaties (Yes/No)", "MPTA (1=wrong,0=good)",
        "PPTA  (1=wrong,0=good)"
    ]

    # Outcome Values columns
    outcome_values = [
        "Total Knee Prosthesis within 2 yrs (Yes/No)", "Total Knee Prosthesis within 10 yrs (Yes/No)",
        "Symptoms (Continuous, 0-100)", "Pain (Continuous, 0-100)", "ADL (Continuous, 0-100)",
        "Sports (Continuous, 0-100)", "Quality of Life (Continuous, 0-100)", "VAS-Satisfaction (1-10) ",
        "Total Knee Prosthesis within 5 yrs (Yes/No)"
    ]

    raw_column_names = df.iloc[0]
    new_column_names = raw_column_names
    for i, column in enumerate(raw_column_names):
        new_column = ""

        if column in patient_characteristics:
            new_column = "patient_" + column
        elif column in fracture_characteristics:
            new_column = "fracture_" + column
        elif column in postop_characteristics:
            new_column = "postop_" + column
        elif column in outcome_values:
            new_column = "outcome_" + column
        else:
            raise ValueError(f"Found unknown column in dataset: {column}, have the column names been modified "
                             f"or have new columns been added?")

        new_column_names[i] = new_column

    df.columns = new_column_names
    df = df.tail(-1)
    return df


def update_data_types(df):
    type_changes = {
        "patient_Age (Continuos)": "float64",
        "patient_Sex (Male/female)": "category",
        "patient_BMI (Continuous": "float64",
        "fracture_Treatment (Conservative/Operative)": "category",
        "fracture_Trauma mechanism (Valgus-flexion, Valgus-extension, Valgus-hyperextension, Varus-flexion, Varus-extension, Varus-Hyperextension)": "category",
        "fracture_Side Fracture fractuur (Right/Left)": "category",
        "fracture_AO/OTA Classification ( 1-6)": "category",
        "fracture_AO/OTA - Severtity": "category",
        "fracture_2D gap (Continuous, mm)": "float64",
        "fracture_2D Step-off (Continuous, mm)": "float64",
        "fracture_3DGap Area": "float64",
        "postop_Condylar width (Continuous, mm)": "float64",
        "postop_Incongruence (Continuous, mm)": "float64",
        "postop_MPTA(Continuous, degree)": "float64",
        "postop_PPTA (Continuous, degree)": "float64",
        "postop_MPTA (1=wrong,0=good)":"bool",
        "postop_PPTA  (1=wrong,0=good)":"bool",
        "patient_Roken": "bool",
        "patient_Diabetes": "bool",
        "patient_Cardiovascular_Disease": "bool",
        "patient_Rheumatoid_Arthritis": "bool",
        "fracture_High_Energy_trauma": "bool",
        "postop_Revision_Surgery": "bool",
        "outcome_Total_Knee_Prothesis_Within_2_Yrs": "bool",
        "outcome_Total_Knee_Prothesis_Within_5_Yrs": "bool",
        "outcome_Total_Knee_Prothesis_Within_10_Yrs": "bool"
    }

    for col, dtype in type_changes.items():
        if dtype == "float64":
            df[col] = pd.to_numeric(df[col], errors='coerce')
        elif dtype == "bool":
            df[col] = df[col].astype("bool")
        elif dtype == "category":
            # Attempting to convert to category. If conversion fails, the value will remain as it was.
            try:
                df[col] = df[col].str.strip()
                # Replace multiple spaces with a single space
                df[col] = df[col].str.replace(r'\s+', ' ')
                df[col] = df[col].astype('category')
            except AttributeError:
                pass
    return df


def update_column(df, column, new_name, func, *args, **kwargs):
    df[new_name] = func(df[column], *args, **kwargs)
    df.drop(columns=[column], inplace=True)
    return df


def normalize_data(df):
    bool_replacement = {"Yes": True, "No": False, "no": False}
    replace_func = lambda c, rep_dict: c.replace(rep_dict)

    update_column(df, 'patient_Roken (Yes/No)', 'patient_Roken', replace_func,
                  bool_replacement)
    update_column(df, 'patient_Diabetes (Yes/No)', 'patient_Diabetes', replace_func,
                  bool_replacement)
    update_column(df, 'patient_Cardiovascular disease (Yes/No)', 'patient_Cardiovascular_Disease',
                  replace_func, bool_replacement)
    update_column(df, 'patient_Rheumatoid Arthritis(Yes/No)', 'patient_Rheumatoid_Arthritis',
                  replace_func, bool_replacement)

    update_column(df, 'fracture_High Energy trauma (Yes/No)', 'fracture_High_Energy_trauma',
                  replace_func, bool_replacement)
    update_column(df, 'fracture_Posterior involvement (Yes/No)', 'fracture_Posterior_Involvement',
                  replace_func, bool_replacement)
    update_column(df, 'fracture_Fracture fibula head (Yes/No)', 'fracture_Fracture_Fibula_Head',
                  replace_func, bool_replacement)
    update_column(df, 'fracture_Cruris Fracture (Yes/No)', 'fracture_Cruris_Fracture',
                  replace_func, bool_replacement)

    update_column(df, 'postop_Revision surgery (Yes/No)', 'postop_Revision_Surgery',
                  replace_func, bool_replacement)
    update_column(df, 'postop_Complicaties (Yes/No)', 'postop_Complicaties', replace_func,bool_replacement)

    update_column(df, 'outcome_Total Knee Prosthesis within 2 yrs (Yes/No)',
                  'outcome_Total_Knee_Prothesis_Within_2_Yrs', replace_func,bool_replacement)
    update_column(df, 'outcome_Total Knee Prosthesis within 5 yrs (Yes/No)',
                  'outcome_Total_Knee_Prothesis_Within_5_Yrs', replace_func,bool_replacement)
    update_column(df, 'outcome_Total Knee Prosthesis within 10 yrs (Yes/No)',
                  'outcome_Total_Knee_Prothesis_Within_10_Yrs', replace_func,bool_replacement)

    return df

def preprocess(data_dir="data", filename="Features-Filled-V3.xlsx"):
    filepath = Path(data_dir).joinpath(filename)
    df = pd.read_excel(filepath)

    df = rename_columns(df)
    df = normalize_data(df)
    df = update_data_types(df)

    preprocessed_path = Path(data_dir).joinpath("preprocessed.csv")
    df.to_csv(preprocessed_path, index=False)


if __name__ == '__main__':
    preprocess()

# Training and Evaluation

In [ ]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (roc_curve, auc, f1_score, confusion_matrix,
                             precision_score, recall_score, ConfusionMatrixDisplay)
from sklearn.calibration import calibration_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
import pickle

from patsy.splines import bs  # Direct use of the spline function

# ------------------------------------------------------------------------------
# 1) INPUT BOUNDARIES (BIOLOGICALLY BACKED)
# ------------------------------------------------------------------------------
input_bounds = {
    "gap":      [0, 100],
    "stepoff":  [0, 100],
    "age":      [0, 120],
    "bmi":      [0, 50]
}

# ------------------------------------------------------------------------------
# 2) MAP FROM COLUMN NAMES TO THE KEYS IN 'input_bounds'
#    (So we can pick the correct [min, max] for each feature.)
# ------------------------------------------------------------------------------
feature_bounds_map = {
    "fracture_2D gap (Continuous, mm)":        "gap",
    "fracture_2D Step-off (Continuous, mm)":   "stepoff",
    "patient_Age (Continuos)":                 "age",
    "patient_BMI (Continuous":                 "bmi"  # note the missing parenthesis in the CSV
}

# ------------------------------------------------------------------------------
# 3) LOAD DATA & SETUP
# ------------------------------------------------------------------------------
file_path = 'preprocessed.csv'
df = pd.read_csv(file_path)

center_column = 'patient_Hospital'  # Adjust if needed
unique_centers = df[center_column].unique()

continuous_features = [
    "fracture_2D gap (Continuous, mm)",
    "fracture_2D Step-off (Continuous, mm)",
    "patient_Age (Continuos)",
    "patient_BMI (Continuous"
]

categorical_features = [
    "patient_Sex (Male/female)",
    "fracture_AO/OTA Classification ( 1-6)",
    "postop_MPTA (1=wrong,0=good)",
    "postop_PPTA  (1=wrong,0=good)"
]

os.makedirs("output", exist_ok=True)
outcomes_path = os.path.join("output", "outcomes.csv")

years = [2, 5]
models = ['logistic', 'forest', 'xgboost']

# ------------------------------------------------------------------------------
# Helper: Convert original [min, max] into scaled [scaled_min, scaled_max]
# ------------------------------------------------------------------------------
def get_scaled_bounds(feature_name, original_bounds, trained_scaler, feat_list):
    """
    For a given feature_name, with known 'original_bounds' (e.g. [0, 100]),
    returns the scaled_min, scaled_max according to the 'trained_scaler'.

    'feat_list' is the list of continuous feature names in the exact order
    used when fitting the scaler, so we can retrieve the correct index.
    """
    col_idx = feat_list.index(feature_name)
    orig_min, orig_max = original_bounds

    mean_ = trained_scaler.mean_[col_idx]
    var_  = trained_scaler.var_[col_idx]
    std_  = np.sqrt(var_)

    scaled_min = (orig_min - mean_) / std_
    scaled_max = (orig_max - mean_) / std_

    return scaled_min, scaled_max

# ------------------------------------------------------------------------------
# MAIN LOOP
# ------------------------------------------------------------------------------
for year in years:
    target = f"outcome_Total_Knee_Prothesis_Within_{year}_Yrs"

    # Drop rows with missing target for this year
    df_year = df.dropna(subset=[target])
    y = df_year[target].astype(bool)

    # --------------------------------------------------------------------------
    # (A) Create and save a single imputer + scaler for this year
    #     since they are not model-dependent.
    # --------------------------------------------------------------------------
    X_all_cont = df_year[continuous_features].copy()
    X_all_cat  = df_year[categorical_features].copy()
    y_all      = y  # Same as above, just for clarity

    # Fit final imputer on *all* data for this year
    imputer_final_year = KNNImputer(n_neighbors=5)
    X_all_cont_imputed_year = imputer_final_year.fit_transform(X_all_cont)

    # Fit final scaler on *all* data for this year
    scaler_final_year = StandardScaler()
    X_all_cont_scaled_year = scaler_final_year.fit_transform(X_all_cont_imputed_year)

    # Save them once (not per model)
    imputer_path = os.path.join("output", f"imputer_{year}.sav")
    scaler_path  = os.path.join("output", f"scaler_{year}.sav")
    pickle.dump(imputer_final_year, open(imputer_path, 'wb'))
    pickle.dump(scaler_final_year, open(scaler_path, 'wb'))

    # This DataFrame will be used later for training final models.
    X_all_cont_scaled_df_year = pd.DataFrame(
        X_all_cont_scaled_year,
        columns=continuous_features,
        index=df_year.index
    )

    # --------------------------------------------------------------------------
    # (B) Cross-validation + model-specific final training, per model
    # --------------------------------------------------------------------------
    for model_name in models:
        print(f"\nModel: {model_name}, Year: {year}")

        # Create a subfolder for each model-year
        run_folder = os.path.join("output", f"{model_name}_{year}")
        os.makedirs(run_folder, exist_ok=True)

        # A prefix we'll use for naming all output files
        prefix = f"{model_name}_{year}"

        # Lists for storing metrics across folds
        all_f1_scores = []
        all_aucs = []
        all_precision = []
        all_recall = []
        all_f1_macro = []
        all_precision_macro = []
        all_recall_macro = []

        cumulative_confusion_matrix = np.zeros((2, 2))
        all_probs = []
        all_y_test = []
        all_y_pred = []

        predictions_list_for_model_year = []
        feature_importances_list_for_model_year = []

        fold_fprs = []
        fold_tprs = []
        fold_roc_aucs = []

        # ===========================================================
        # 1) CROSS VALIDATION (LEAVE-ONE-CENTER-OUT) FOR EVALUATION
        # ===========================================================
        for test_center in unique_centers:
            train_mask = (df_year[center_column] != test_center)
            test_mask  = (df_year[center_column] == test_center)

            X_train_df, X_test_df = df_year.loc[train_mask], df_year.loc[test_mask]
            y_train, y_test       = y.loc[train_mask], y.loc[test_mask]

            if len(y_test) == 0:
                continue

            # 1A) Split continuous vs. categorical
            X_train_cont = X_train_df[continuous_features].copy()
            X_test_cont  = X_test_df[continuous_features].copy()
            X_train_cat  = X_train_df[categorical_features].copy()
            X_test_cat   = X_test_df[categorical_features].copy()

            # 1B) Impute + scale *for this CV fold only*
            imputer_fold = KNNImputer(n_neighbors=5)
            X_train_cont_imputed = imputer_fold.fit_transform(X_train_cont)
            X_test_cont_imputed  = imputer_fold.transform(X_test_cont)

            scaler_fold = StandardScaler()
            X_train_cont_scaled = scaler_fold.fit_transform(X_train_cont_imputed)
            X_test_cont_scaled  = scaler_fold.transform(X_test_cont_imputed)

            X_train_cont_scaled_df = pd.DataFrame(
                X_train_cont_scaled,
                columns=continuous_features,
                index=X_train_df.index
            )
            X_test_cont_scaled_df = pd.DataFrame(
                X_test_cont_scaled,
                columns=continuous_features,
                index=X_test_df.index
            )

            # 1C) Build final train/test features depending on model
            if model_name == 'logistic':
                # Create splines with boundary knots (scaled) for each feature
                X_spline_list_train = []
                X_spline_list_test = []

                for cfeat in continuous_features:
                    # Skip if not enough data
                    cvals_train = X_train_cont_scaled_df[cfeat].dropna()
                    if len(cvals_train) < 4:
                        print(f"[Fold CV] Dropping {cfeat} - insufficient data.")
                        continue

                    # Retrieve original bounds + convert to scaled space
                    bounds_key = feature_bounds_map[cfeat]
                    orig_bounds = input_bounds[bounds_key]

                    scaled_left, scaled_right = get_scaled_bounds(
                        feature_name=cfeat,
                        original_bounds=orig_bounds,
                        trained_scaler=scaler_fold,
                        feat_list=continuous_features
                    )

                    # Internal knots: 25, 50, 75 percentiles (from training data)
                    knots = np.percentile(cvals_train, [25, 50, 75])

                    # -- Train set splines --
                    train_values = X_train_cont_scaled_df[cfeat].values
                    train_bs = bs(
                        train_values,
                        knots=knots,
                        degree=3,
                        include_intercept=False,
                        lower_bound=scaled_left,
                        upper_bound=scaled_right
                    )
                    spline_col_names = [f"{cfeat}_spline_{i}" for i in range(train_bs.shape[1])]
                    spline_train = pd.DataFrame(
                        train_bs,
                        columns=spline_col_names,
                        index=X_train_cont_scaled_df.index
                    )
                    X_spline_list_train.append(spline_train)

                    # -- Test set splines --
                    test_values = X_test_cont_scaled_df[cfeat].values
                    test_bs = bs(
                        test_values,
                        knots=knots,
                        degree=3,
                        include_intercept=False,
                        lower_bound=scaled_left,
                        upper_bound=scaled_right
                    )
                    spline_test = pd.DataFrame(
                        test_bs,
                        columns=spline_col_names,
                        index=X_test_cont_scaled_df.index
                    )
                    X_spline_list_test.append(spline_test)

                if X_spline_list_train:
                    X_spline_all_train = pd.concat(X_spline_list_train, axis=1)
                    X_spline_all_test  = pd.concat(X_spline_list_test, axis=1)
                else:
                    X_spline_all_train = pd.DataFrame(index=X_train_df.index)
                    X_spline_all_test  = pd.DataFrame(index=X_test_df.index)

                X_train_final = pd.concat([X_train_cat, X_spline_all_train], axis=1)
                X_test_final  = pd.concat([X_test_cat,  X_spline_all_test], axis=1)

            else:
                # forest/xgboost => just scaled + cat (no spline transformations)
                X_train_final = pd.concat([X_train_cont_scaled_df, X_train_cat], axis=1)
                X_test_final  = pd.concat([X_test_cont_scaled_df,  X_test_cat], axis=1)

            # 1D) Choose model
            if model_name == 'logistic':
                base_model = LogisticRegression()
            elif model_name == 'forest':
                base_model = RandomForestClassifier()
            elif model_name == 'xgboost':
                base_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

            # Nested validation for threshold selection
            X_train_nested, X_val_nested, y_train_nested, y_val_nested = train_test_split(
                X_train_final, y_train, test_size=0.2, random_state=42
            )
            model_fold = base_model
            model_fold.fit(X_train_nested, y_train_nested)

            thresholds = np.linspace(0.1, 0.9, 9)
            y_proba_val = model_fold.predict_proba(X_val_nested)[:, 1]
            best_threshold = 0
            best_f1 = 0
            for threshold in thresholds:
                y_pred_threshold = (y_proba_val >= threshold).astype(int)
                f1_tmp = f1_score(y_val_nested, y_pred_threshold)
                if f1_tmp > best_f1:
                    best_f1 = f1_tmp
                    best_threshold = threshold

            # Retrain on full training set for this fold
            model_fold.fit(X_train_final, y_train)
            y_proba = model_fold.predict_proba(X_test_final)[:, 1]
            y_pred = (y_proba >= best_threshold).astype(int)

            # Metrics
            f1       = f1_score(y_test, y_pred)
            precision= precision_score(y_test, y_pred)
            recall   = recall_score(y_test, y_pred)

            f1_macro       = f1_score(y_test, y_pred, average='macro')
            precision_macro= precision_score(y_test, y_pred, average='macro')
            recall_macro   = recall_score(y_test, y_pred, average='macro')

            fpr, tpr, _ = roc_curve(y_test, y_proba)
            roc_auc     = auc(fpr, tpr)

            all_f1_scores.append(f1)
            all_aucs.append(roc_auc)
            all_precision.append(precision)
            all_recall.append(recall)
            all_f1_macro.append(f1_macro)
            all_precision_macro.append(precision_macro)
            all_recall_macro.append(recall_macro)

            all_probs.extend(y_proba)
            all_y_test.extend(y_test)
            all_y_pred.extend(y_pred)

            fold_confusion_matrix = confusion_matrix(y_test, y_pred)
            cumulative_confusion_matrix += fold_confusion_matrix

            # Predictions table
            df_pred_center = pd.DataFrame({
                "patient_Patient number": X_test_df["patient_Patient number"],
                "center": test_center,
                "year": year,
                "model": model_name,
                "true_label": y_test.values,
                "predicted_proba": y_proba,
                "predicted_label": y_pred
            })
            predictions_list_for_model_year.append(df_pred_center)

            # Feature importances
            if model_name == 'logistic':
                importances = model_fold.coef_[0]
            else:
                importances = model_fold.feature_importances_

            final_feature_names = X_train_final.columns.tolist()
            df_importance = pd.DataFrame({
                'feature': final_feature_names,
                'importance': importances
            })
            df_importance['center'] = test_center
            df_importance['model'] = model_name
            df_importance['year'] = year

            feature_importances_list_for_model_year.append(df_importance)

            fold_fprs.append(fpr)
            fold_tprs.append(tpr)
            fold_roc_aucs.append(roc_auc)

        # ===========================================================
        # 2) AFTER CROSS-VAL, PLOT AND TRAIN A FINAL MODEL
        # ===========================================================
        if len(all_f1_scores) > 0:
            mean_f1   = np.mean(all_f1_scores)
            mean_auc  = np.mean(all_aucs)
            mean_prec = np.mean(all_precision)
            mean_rec  = np.mean(all_recall)

            mean_f1_macro        = np.mean(all_f1_macro)
            mean_precision_macro = np.mean(all_precision_macro)
            mean_recall_macro    = np.mean(all_recall_macro)

            print(f"Mean F1: {mean_f1:.2f}")
            print(f"Mean AUC: {mean_auc:.2f}")
            print(f"Mean Precision: {mean_prec:.2f}")
            print(f"Mean Recall: {mean_rec:.2f}")
            print(f"Mean F1 (macro): {mean_f1_macro:.2f}")
            print(f"Mean Precision (macro): {mean_precision_macro:.2f}")
            print(f"Mean Recall (macro): {mean_recall_macro:.2f}")

            # Confusion Matrix
            cm_filename = os.path.join(run_folder, f"{prefix}_confusion_matrix.png")
            ConfusionMatrixDisplay(confusion_matrix=cumulative_confusion_matrix).plot(values_format='g')
            plt.title(f"Confusion Matrix: {model_name}, {year}-Year")
            plt.savefig(cm_filename)
            plt.show()

            # Bootstrapping for AUC
            n_bootstraps = 100
            rng = np.random.RandomState(42)
            boot_aucs = []
            all_y_test_arr = np.array(all_y_test)
            all_probs_arr  = np.array(all_probs)
            for i in range(n_bootstraps):
                indices = rng.randint(0, len(all_y_test_arr), len(all_y_test_arr))
                # If sample is all one class, skip
                if len(np.unique(all_y_test_arr[indices])) < 2:
                    continue
                fpr_b, tpr_b, _ = roc_curve(all_y_test_arr[indices], all_probs_arr[indices])
                roc_auc_b = auc(fpr_b, tpr_b)
                boot_aucs.append(roc_auc_b)

            if len(boot_aucs) > 0:
                lower_ci = np.percentile(boot_aucs, 2.5)
                upper_ci = np.percentile(boot_aucs, 97.5)
                print(f"Bootstrap AUC Mean: {np.mean(boot_aucs):.2f}, "
                      f"95% CI: ({lower_ci:.2f}-{upper_ci:.2f})")

            # Calibration curve
            cal_curve_filename = os.path.join(run_folder, f"{prefix}_calibration_curve.png")
            prob_true, prob_pred = calibration_curve(all_y_test, all_probs, n_bins=10)
            plt.figure()
            plt.plot(prob_pred, prob_true, marker='o', label='Calibration')
            plt.plot([0,1],[0,1], 'k--', label='Perfectly calibrated')
            plt.title(f"Calibration Curve: {model_name}, {year}-Year")
            plt.xlabel('Predicted Probability')
            plt.ylabel('Fraction of Positives')
            plt.legend()
            plt.savefig(cal_curve_filename)
            plt.show()

            # Summarize cross‐val run
            run_data = {
                "name": f"{model_name}_{year}",
                "auc": round(mean_auc, 3),
                "f1": round(mean_f1, 3),
                "recall": round(mean_rec, 3),
                "precision": round(mean_prec, 3),
                "f1_macro": round(mean_f1_macro, 3),
                "recall_macro": round(mean_recall_macro, 3),
                "precision_macro": round(mean_precision_macro, 3),
                "bootstrap_auc_mean": round(np.mean(boot_aucs), 3) if len(boot_aucs) > 0 else np.nan,
                "bootstrap_auc_ci_lower": round(lower_ci, 3) if len(boot_aucs) > 0 else np.nan,
                "bootstrap_auc_ci_upper": round(upper_ci, 3) if len(boot_aucs) > 0 else np.nan,
            }

            try:
                outcome = pd.read_csv(outcomes_path)
                outcome = pd.concat([outcome, pd.DataFrame.from_dict([run_data])], ignore_index=True)
                outcome.to_csv(outcomes_path, index=False)
            except FileNotFoundError:
                pd.DataFrame.from_dict([run_data]).to_csv(outcomes_path, index=False)

            # Save predictions
            if predictions_list_for_model_year:
                df_all_preds_this_model_year = pd.concat(predictions_list_for_model_year, ignore_index=True)
                df_all_preds_this_model_year.to_csv(
                    os.path.join(run_folder, f"{prefix}_predictions.csv"),
                    index=False
                )

            # Save feature importances
            if feature_importances_list_for_model_year:
                df_feature_importances = pd.concat(feature_importances_list_for_model_year, ignore_index=True)
                df_feature_importances.to_csv(
                    os.path.join(run_folder, f"{prefix}_feature_importances.csv"),
                    index=False
                )

            # ============================================
            # 3) TRAIN THE FINAL MODEL ON ALL df_year DATA
            #    (using the year-level imputer/scaler)
            # ============================================
            print(f"Training final {model_name} model on *all* data (year={year})...")

            # We already have X_all_cont_scaled_df_year, X_all_cat, y_all
            # from the year-level imputer/scaler
            knots_dict_final = {}
            if model_name == 'logistic':
                # Build spline columns using the already-scaled data
                X_spline_list_all = []
                for cfeat in continuous_features:
                    cvals_all = X_all_cont_scaled_df_year[cfeat].dropna()
                    if len(cvals_all) < 4:
                        print(f"(Final) Dropping {cfeat} due to insufficient data.")
                        continue

                    # Retrieve original [min, max] -> scaled
                    bounds_key = feature_bounds_map[cfeat]
                    b_left_orig, b_right_orig = input_bounds[bounds_key]
                    scaled_left_final, scaled_right_final = get_scaled_bounds(
                        feature_name=cfeat,
                        original_bounds=[b_left_orig, b_right_orig],
                        trained_scaler=scaler_final_year,
                        feat_list=continuous_features
                    )

                    # Internal knots from entire data distribution
                    knots_final = np.percentile(cvals_all, [25, 50, 75])

                    # Save them for production usage
                    knots_dict_final[cfeat] = {
                        "internal_knots": knots_final.tolist(),
                        "boundary_knots_original": [float(b_left_orig), float(b_right_orig)],
                        "boundary_knots_scaled": [float(scaled_left_final), float(scaled_right_final)]
                    }

                    all_values = X_all_cont_scaled_df_year[cfeat].values
                    spline_all = bs(
                        all_values,
                        knots=knots_final,
                        degree=3,
                        include_intercept=False,
                        lower_bound=scaled_left_final,
                        upper_bound=scaled_right_final
                    )
                    spline_col_names = [f"{cfeat}_spline_{i}" for i in range(spline_all.shape[1])]
                    spline_all_df = pd.DataFrame(
                        spline_all,
                        columns=spline_col_names,
                        index=X_all_cont_scaled_df_year.index
                    )
                    X_spline_list_all.append(spline_all_df)

                if X_spline_list_all:
                    X_spline_all_final = pd.concat(X_spline_list_all, axis=1)
                else:
                    X_spline_all_final = pd.DataFrame(index=df_year.index)

                X_final_all = pd.concat([X_all_cat, X_spline_all_final], axis=1)
                final_model = LogisticRegression()

            elif model_name == 'forest':
                # Random Forest uses scaled continuous + cat (no spline)
                X_final_all = pd.concat([X_all_cont_scaled_df_year, X_all_cat], axis=1)
                final_model = RandomForestClassifier()

            elif model_name == 'xgboost':
                # XGBoost uses scaled continuous + cat (no spline)
                X_final_all = pd.concat([X_all_cont_scaled_df_year, X_all_cat], axis=1)
                final_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

            print("X_final_all shape:", X_final_all.shape)
            print("y_all shape:", y_all.shape)

            # Fit final model
            final_model.fit(X_final_all, y_all)

            # Save final model (named as <model>_<year>.sav)
            model_filename = os.path.join(run_folder, f"{prefix}.sav")
            pickle.dump(final_model, open(model_filename, 'wb'))

            # Save the final spline boundary/knots data (only relevant for logistic)
            # For forest/xgboost, we keep it empty or skip.
            knots_path = os.path.join(run_folder, f"{prefix}_knots.json")
            with open(knots_path, "w") as f:
                json.dump(knots_dict_final, f, indent=2)

            print(f"Saved final {model_name} for {year}-year to {run_folder}.\n")
        # end if cross-val had folds
    # end model loop
# end year loop


# Download Output (Google Colab)

In [ ]:
try:
    # Check if running in Google Colab
    import google.colab

    # 1. Zip a specific folder (replace 'output' with your folder name).
    !zip -r output.zip output

    # 2. Download the zip file.
    from google.colab import files
    files.download('output.zip')

except ImportError:
    print("This script is intended to run on Google Colab.")
